In [3]:
using JuLIP, PoSH, IPFitting, HDF5, LinearAlgebra
using JuLIP.MLIPs: IPSuperBasis
using JSON
using PolyPairPots
include("./reglsq.jl")

┌ Info: Precompiling IPFitting [3002bd4c-79e4-52ce-b924-91256dde4e52]
└ @ Base loading.jl:1186
┌ Info: Precompiling ASE [51974c44-a7ed-5088-b8be-3e78c8ba416c]
└ @ Base loading.jl:1186


reglsq (generic function with 1 method)

### Reading in the unregularised potential

In [11]:
D = load_json("PoSH_Si_fit_unreg.json")
lsqinfo = D["info"]

Dict{String,Any} with 11 entries:
  "c"                 => Any[7.95992e13, -8.83438e13, 7.63694e13, -5.16525e13, …
  "dbpath"            => "Si"
  "Ibasis"            => Any[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  752, 753, 754, 7…
  "weights"           => Dict{String,Any}("amorph"=>Dict{String,Any}("V"=>0.25,…
  "kappa"             => 6.65714e36
  "solver"            => "qr"
  "juliaversion"      => "Julia Version 1.1.0\nCommit 80516ca202 (2019-01-21 21…
  "regularisers"      => Any[]
  "IPFitting_version" => Dict{String,Any}("name"=>"IPFitting","uuid"=>"3002bd4c…
  "E0"                => -158.545
  "errors"            => Dict{String,Any}("relrmse"=>Dict{String,Any}("sh"=>Dic…

### Reading in the configurations

In [12]:
cfgs = IPFitting.Data.read_xyz("Si.xyz")[1:50:end];

Reading in Si.xyz ...
Processing data ...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


### Setting up the basis (identical to the fit for the unregularised potentail)

In [13]:
r0 = rnn(:Si)
rcut_in = 0.68*r0
rcut_out = 5.2       

trans = PoSH.PolyTransform(2, r0)
cutf = PoSH.PolyCutoff1s(2, 0.0, rcut_out)

N = 4
polydeg = 12
wL = 1.4
chc = 0.1

shipbasis = SHIPBasis(SparseSHIP(:Si, N, polydeg; wL = wL, chc = chc), trans, cutf);

### Performing the regularisation

`τ` is a target expected error and generally 1.1/1.2/1.3 the rmse error of the unregularised potential. The regularisation will in this sense sacrifice 10%/20%/30% RMSE accuracy at the expense of regularisation

In [14]:
# Load the QR factorisation again
fid = h5open("Si_qr.h5","r")
R = read(fid, "R")
y = read(fid, "y")
η0  = read(fid, "rmse")
close(fid)

τ = 1.50 * η0
Γ = Matrix(1.0*I, length(shipbasis), length(shipbasis))


c = reglsq(Γ = Γ, R = R, z=y, τ = τ, η0 = η0, )
@show norm(c)

using JuLIP.MLIPs: combine, SumIP

E0 = -158.54496821

Vref = MOneBody(Dict(:Si => E0))

shipIP = SumIP(Vref, combine(shipbasis, c));

┌ Info: `reglsq` : solve regularised least squares
└ @ Main /Users/Cas/gits/JuliaMolSim_Docs/reglsq.jl:17
┌ Info: found bracket, starting bisection
└ @ Main /Users/Cas/gits/JuliaMolSim_Docs/reglsq.jl:40


norm(c) = 718009.1467196249


┌ Error: bracket has become too small; returning current solutions,
│ if it is unsatisfactory, then please reduce the
│ `abstol` parameter.
└ @ Main /Users/Cas/gits/JuliaMolSim_Docs/reglsq.jl:55


### Unregularised table of errors

In [18]:
rmse_table(D["info"]["errors"])

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ RMSE                                                               ┃
┣━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┫
┃ config type  ┃     E [meV]     │     F [eV/A]    │     V [meV]     ┃
┠──────────────╂────────┬────────┼────────┬────────┼────────┬────────┨
┃           sh ┃  38.48 ┊ 0.024% │  0.121 ┊ 35.53% │  171.1 ┊ 18.75% ┃
┃      vacancy ┃   4.33 ┊ 0.003% │  0.155 ┊ 20.31% │   69.8 ┊ 40.36% ┃
┃          sp2 ┃   1.08 ┊ 0.001% │  1.280 ┊ 199.34% │    NaN ┊   NaN% ┃
┃          fcc ┃  12.77 ┊ 0.008% │  0.271 ┊ 130.91% │  307.5 ┊ 151.95% ┃
┃           bt ┃   2.74 ┊ 0.002% │  0.124 ┊ 45.83% │  112.2 ┊  8.43% ┃
┃ crack.._1-10 ┃   3.35 ┊ 0.002% │  0.310 ┊ 54.22% │   85.5 ┊ 28.27% ┃
┃ isola.._atom ┃   0.00 ┊ 0.000% │  0.000 ┊   NaN% │    NaN ┊   NaN% ┃
┃          dia ┃   4.62 ┊ 0.003% │  0.090 ┊ 24.86% │   37.3 ┊  2.83% ┃
┃    divacancy ┃   1.65 ┊ 0.001% │  0.181 ┊ 18.21% │    NaN ┊   NaN% ┃
┃ i

### Calculating the errors for the regularised potential

`τ` is a target expected error and generally 1.1/1.2/1.3 the rmse error of the unregularised potential. The regularisation will in this sense sacrifice 10%/20%/30% RMSE accuracy at the expense of regularisation

In [16]:
add_fits!(shipIP, cfgs; fitkey="fit") #use add_fits! also crashse!!
rmse_test, rmserel_test = rmse(cfgs; fitkey="fit")
lsqinfo["errors"] = Dict("rmse" => rmse_test, "relrmse" => rmserel_test)

rmse_table(lsqinfo["errors"])

save_json("PoSH_Si_fit_reg.json", Dict("IP" => Dict(shipIP), "info" => lsqinfo))

Add Fit info to configs in serial


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ RMSE                                                               ┃
┣━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━┫
┃ config type  ┃     E [meV]     │     F [eV/A]    │     V [meV]     ┃
┠──────────────╂────────┬────────┼────────┬────────┼────────┬────────┨
┃           sh ┃  38.48 ┊ 0.024% │  0.121 ┊ 35.53% │  171.1 ┊ 18.75% ┃
┃      vacancy ┃   4.33 ┊ 0.003% │  0.155 ┊ 20.31% │   69.8 ┊ 40.36% ┃
┃          sp2 ┃   1.08 ┊ 0.001% │  1.280 ┊ 199.34% │    NaN ┊   NaN% ┃
┃          fcc ┃  12.77 ┊ 0.008% │  0.271 ┊ 130.91% │  307.5 ┊ 151.95% ┃
┃           bt ┃   2.74 ┊ 0.002% │  0.124 ┊ 45.83% │  112.2 ┊  8.43% ┃
┃ crack.._1-10 ┃   3.35 ┊ 0.002% │  0.310 ┊ 54.22% │   85.5 ┊ 28.27% ┃
┃ isola.._atom ┃   0.00 ┊ 0.000% │  0.000 ┊   NaN% │    NaN ┊   NaN% ┃
┃          dia ┃   4.62 ┊ 0.003% │  0.090 ┊ 24.86% │   37.3 ┊  2.83% ┃
┃    divacancy ┃   1.65 ┊ 0.001% │  0.181 ┊ 18.21% │    NaN ┊   NaN% ┃
┃ i

┌ Info: Elapsed: 1.4s
└ @ IPFitting.Tools /Users/Cas/.julia/packages/IPFitting/4xRUd/src/tools.jl:68
